In [2]:
%cd /home/sasedov/StyleDomain/SimilarDomains

/home/sasedov/StyleDomain/SimilarDomains


/home/sasedov/.conda/envs/StyleDomain-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import PIL
import random

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor
from omegaconf import OmegaConf
from core.utils.common import load_clip, mixing_noise
from core.utils.example_utils import (
    Inferencer, to_im, vstack_with_lines, hstack_with_lines, insert_image,
    project_e4e, project_restyle_psp, project_fse_without_image_generation, read_img
)
from core.utils.image_utils import construct_paper_image_grid
from core.utils.reading_weights import read_weights
from core.uda_models import OffsetsTunningGenerator

from pathlib import Path
from collections import defaultdict

from examples.draw_util import weights, set_seed, morph_g_ema, IdentityEditor, StyleEditor

from core.utils.example_utils import read_fse_config
scale, scale_mode, idx_k, n_styles, enc_residual, enc_residual_coeff, resnet_layers, stride = read_fse_config('001')
print(scale, scale_mode, idx_k, n_styles, enc_residual, enc_residual_coeff, resnet_layers, stride)

2 bilinear 5 18 False False [4, 5, 6] (2, 2)


In [4]:
def stack_rows(rows, skip_horiz=10, skip_vertical=15):
    final_image = [
        hstack_with_lines(row_stack, skip_horiz) for row_stack in rows
    ]

    final_image = vstack_with_lines(final_image, skip_vertical)
    return PIL.Image.fromarray(final_image)


style_to_editor = {
    d: StyleEditor(read_weights(weights[d])) for d in weights if d not in ['horse', 'car', 'ffhq', 'cat', 'church', 'to_metfaces', 'to_afhqcat', 'to_afhqdog', 'to_mega']
}

style_to_editor['original'] = IdentityEditor()
device = 'cuda:0'

In [ ]:
# FFHQ-SMALL DATASET

from test_dataset import SimpleImageDataset, align_image_dataset
from torch.utils.data import DataLoader

dataset_path = '/home/sasedov/StyleDomain/faces_dataset_small/'
# aligned_dataset_path = '/home/sasedov/StyleDomain/aligned_faces_dataset_small/'

e4e_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ]
)
fse_transform = transforms.Compose(
    [
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]
)

# align_image_dataset(dataset_path, aligned_dataset_path)
e4e_ffhq_small_dataset = SimpleImageDataset(dataset_path, e4e_transform)
fse_ffhq_small_dataset = SimpleImageDataset(dataset_path, fse_transform)

e4e_ffhq_small_dataloader = DataLoader(e4e_ffhq_small_dataset, batch_size=32, shuffle=False)
fse_ffhq_small_dataloader = DataLoader(fse_ffhq_small_dataset, batch_size=32, shuffle=False)

In [ ]:
# READ IMAGES, GET W-S AND FEATURE-MAPS

# encoders' latents for ffhq small
# do we want to compare generator feature maps with FSE feature maps too ?

from contextlib import redirect_stdout
from tqdm import tqdm

import io
import os

ws_e4e = []
ws_fse = []
fse_features_list = []
orig_features_list = []
filenames = []

f_stdout = io.StringIO()

gan_domain = 'ffhq'
s_domain = 'jojo'

ckpt = read_weights(weights[s_domain])
ckpt_ffhq = {'sg2_params': ckpt['sg2_params']}
ckpt_ffhq['sg2_params']['checkpoint_path'] = weights[gan_domain]

model = Inferencer(ckpt, device)

with torch.no_grad():
    with redirect_stdout(f_stdout):
        for e4e_batch, fse_batch in tqdm(zip(e4e_ffhq_small_dataloader, fse_ffhq_small_dataloader), total=len(fse_ffhq_small_dataloader)):
            
            im_e4e_batch, names_e4e_batch = e4e_batch["image"], e4e_batch["filename"]
            im_fse_batch, names_fse_batch = fse_batch["image"], fse_batch["filename"]
            
            assert names_e4e_batch == names_fse_batch, f'{names_e4e_batch}, {names_fse_batch}' 
            print(names_e4e_batch)
            for name in names_e4e_batch:
                filenames.append(name)
            
            w_e4e_batch = project_e4e(
                im_e4e_batch,
                'pretrained/e4e_ffhq_encode.pt',
                use_transforms=False
            )[1]

            empty_img, w_fse_batch, fse_features_batch = project_fse_without_image_generation(
                im_fse_batch,
                model_path='pretrained/143_enc.pth',
                fse_config_name='001',
                arcface_model_path='pretrained/backbone.pth',
                stylegan_model_path='pretrained/StyleGAN2/stylegan2-ffhq-config-f.pt',
                use_transforms=False
            )

            features_batch = []
            
            for w_fse in w_fse_batch:
                print("fse_latents.shape:", w_fse.unsqueeze(0).shape)
                kwargs = {
                    "latents": [w_fse.unsqueeze(0)],
                    "input_is_latent": True,
                    "features_in": None,
                    "shift_with_generator_feature_map": False,
                    "return_features": True,
                    "return_generator_features": True
                }
                imgs, features = model(**kwargs)
                print(len(imgs), len(features))
                print(len(features[0]), len(features[1]))
                
                print(features[0][5].shape, features[1][5].shape)
                print(fse_features_batch[0][5].shape)
                features_batch.append((features[0][5].detach(), features[1][5].detach()))
            
            print(f"Orig features' norm: {features_batch[0][0].norm().detach().cpu().numpy()}")
            print(f"Domain features' norm: {features_batch[0][1].norm().detach().cpu().numpy()}")
            print(f"Norm of domain difference: {(features_batch[0][1] - features_batch[0][0]).norm().detach().cpu().numpy()}")
            
            print(f"FSE features' norm: {fse_features_batch[0][5].norm().detach().cpu().numpy()}")
            print(f"Norm of FSE difference: {(fse_features_batch[0][5] - features_batch[0][0]).norm().detach().cpu().numpy()}")
            print(f"Norm of FSE difference with domain shift: {(fse_features_batch[0][5] - features_batch[0][1]).norm().detach().cpu().numpy()}")

            for w_e4e, w_fse, fse_features, orig_features in zip(w_e4e_batch, w_fse_batch, fse_features_batch, features_batch):
                ws_e4e.append(w_e4e)
                ws_fse.append(w_fse)
                fse_features_list.append(fse_features)
                orig_features_list.append(orig_features)

In [ ]:
for i, (w_e4e, w_fse) in enumerate(zip(ws_e4e, ws_fse)):
    ws_e4e[i] = w_e4e.unsqueeze(0)
    ws_fse[i] = w_fse.unsqueeze(0)

In [ ]:
# ws_e4e = torch.cat(ws_e4e, dim=0)
# ws_fse = torch.cat(ws_fse, dim=0)

fse_features_tensor = []
skipped_indices = []
for i, fse_features in enumerate(fse_features_list):
    if fse_features is None:
        print(i, ws_e4e[i].shape, ws_fse[i].shape)
        skipped_indices.append(i)
    else:
        fse_features_tensor.append(fse_features[5])

fse_features_tensor = torch.cat(fse_features_tensor, dim=0)


orig_features_tensor_src = []
orig_features_tensor_trg = []

for i, orig_features in enumerate(orig_features_list):
    orig_features_tensor_src.append(orig_features[0])
    orig_features_tensor_trg.append(orig_features[1])

orig_features_tensor_src = torch.cat(orig_features_tensor_src, dim=0)
orig_features_tensor_trg = torch.cat(orig_features_tensor_trg, dim=0)

In [ ]:
dsfsfs 
torch.save(ws_e4e, f"ws_e4e_tensor.pt")
torch.save(ws_fse, f"ws_fse_tensor.pt")
torch.save(fse_features_tensor, f"fse_features_tesnor.pt")
torch.save(orig_features_tensor_src, f"orig_features_tensor_src.pt")
torch.save(orig_features_tensor_trg, f"orig_features_tensor_trg.pt")

with open('latent_corr_exp_filenames.txt', 'w') as f:
    for name in filenames:
        f.write(f"{name}\n")